Нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 
Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

### Описание данных

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

#### Для решения задачи использовалась модель BERT-bert-base-uncased, English https://huggingface.co/transformers/pretrained_models.html

# 1. Подготовка

In [1]:
import numpy as np
import pandas as pd
import torch
import transformers
from tqdm import notebook
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import gc
from sklearn.metrics import f1_score
import tensorflow as tf
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


In [2]:
df = pd.read_csv('toxic_comments.csv')

In [3]:
df.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


#### Подготовим признаки

In [4]:
#Для уменьшения объема используемой памяти приведем целевой признак к типу int8
df['toxic'] = df['toxic'].astype('int8')

In [5]:
#Приведем текст датасета к нижнему регистру
df['text'] = df['text'].apply(lambda x: x.lower())

#### Токенизируем текст при помощи BertTokenizerFast

In [6]:
tokenizer = transformers.BertTokenizerFast.from_pretrained('bert-base-uncased')
#tokenizer = transformers.AlbertTokenizer.from_pretrained('albert-base-v2')

In [7]:
tokenized = df['text'].apply(
    lambda x: tokenizer.encode(x, add_special_tokens=True))

#### Оценим максимальную длину токена

In [8]:
max_len = max(map(len, tokenized))
max_len

4950

#### В датасете имеются очень длинные токены, модель BERT требует использовать токены менее длиной менее 512. Исключим из датасета длинные токены.

In [9]:
len_tok = tokenized.apply(lambda x: len(x))
print('Из датасета необходимо удалить',len_tok[len_tok>512].count(), 'значений')
index_to_drop = len_tok[len_tok>512].index

Из датасета необходимо удалить 3523 значений


In [10]:
df = df.drop(index_to_drop).reset_index(drop=True)
tokenized = tokenized.drop(index_to_drop).reset_index(drop=True)

In [11]:
tokenized

0         [101, 7526, 2339, 1996, 10086, 2015, 2081, 210...
1         [101, 1040, 1005, 22091, 2860, 999, 2002, 3503...
2         [101, 4931, 2158, 1010, 1045, 1005, 1049, 2428...
3         [101, 1000, 2062, 1045, 2064, 1005, 1056, 2191...
4         [101, 2017, 1010, 2909, 1010, 2024, 2026, 5394...
                                ...                        
156043    [101, 1000, 1024, 1024, 1024, 1024, 1024, 1998...
156044    [101, 2017, 2323, 2022, 14984, 1997, 4426, 200...
156045    [101, 13183, 6290, 26114, 1010, 2045, 2015, 20...
156046    [101, 1998, 2009, 3504, 2066, 2009, 2001, 2941...
156047    [101, 1000, 1998, 1012, 1012, 1012, 1045, 2428...
Name: text, Length: 156048, dtype: object

#### Выделим целевой признак и удалим из памяти датасет, для уменьшения объема требуемой памяти

In [12]:
target = df['toxic']
del df

#### Проверим сбалансированность классов

In [13]:
target.value_counts()

0    140169
1     15879
Name: toxic, dtype: int64

#### Выборка не сбалансирована. Для ускорения обучения моделей и улучшения предсказаний необходимо будет провести downsampling

In [14]:
def downsampling(features, target, fraction):
    """
    Функция проводит downsampling выборки (признаков с целевым признаком равным 0)
    На выходе дает перемешанные признаки и целевой признак. Для удобства, индексы будут сброшены
    """
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])
    
    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=12345)
    
    return features_downsampled.reset_index(drop=True), target_downsampled.reset_index(drop=True)

#### Проведем downsampling

In [15]:
tokenized_downsampled, target_downsampled = downsampling(tokenized, target, 0.15)

#### Удалим более не нужные серии

In [16]:
del tokenized, target

#### Проверим максимальную длину токена

In [17]:
max_len = max(map(len, tokenized_downsampled))
max_len

512

#### Добавим нули к токенам длиной меньшим чем max_len, чтобы преобразовать токены в вектора.

In [18]:
def add_zeros(x):
    """
    Функция добавляет нули к строкам массива меньшим по длинне чем max_len
    """
    x = np.array(x + [0]*(max_len - len(x)), dtype='int16')
    return x

In [19]:
tokenized_downsampled = tokenized_downsampled.apply(add_zeros).to_numpy()

padded = np.zeros((len(tokenized_downsampled), len(tokenized_downsampled[0])), dtype='int16' )

for i in range(len(tokenized_downsampled)):
    padded[i,:] = tokenized_downsampled[i]

In [20]:
del tokenized_downsampled

#### Создадим маску

In [21]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask = attention_mask.astype('int16')

#### Выберем модель для эмбеддинга, конфигурация модели - по умолчанию

In [23]:
model = transformers.BertModel.from_pretrained('bert-base-uncased')
#model = transformers.AlbertForMaskedLM.from_pretrained('albert-base-v2')
model.config

BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

In [24]:
padded.shape

(36904, 512)

In [25]:
attention_mask.shape

(36904, 512)

#### Для эмбеддинга будем использовать GPU, если это возможно

In [26]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [27]:
model = model.to(device)

#### Для освобождения памяти перед обучением запустим сборщик мусора

In [28]:
gc.collect()

355

#### Создадим эмбеддинги. Создание на локальной машине заняло 1 час 10 минут.

In [29]:
batch_size = 20
embeddings = []
for i in notebook.tqdm(range(padded.shape[0] // (batch_size))):
        batch = torch.cuda.LongTensor(padded[batch_size*i:batch_size*(i+1)]) 
        attention_mask_batch = torch.cuda.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
        
        with torch.no_grad():
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)
        
        embeddings.append(batch_embeddings[0][:,0,:].cpu().numpy())

In [30]:
embeddings

[array([[ 0.26013154, -0.20489092,  0.13780355, ..., -0.04069155,
          0.39130023,  0.6664716 ],
        [-0.2555647 ,  0.05485541, -0.05909117, ..., -0.42201525,
          0.29885727,  0.77606463],
        [-0.19205336,  0.05815222,  0.02054871, ..., -0.12545663,
          0.42757124,  0.43673858],
        ...,
        [ 0.17834   , -0.11203216,  0.15927605, ..., -0.11390832,
          0.3793795 ,  0.6621577 ],
        [-0.21299556, -0.00418272,  0.4547703 , ..., -0.5481356 ,
          0.11202361,  0.5528518 ],
        [ 0.51542974,  0.17245963, -0.05093499, ..., -0.3282208 ,
          0.8195886 ,  0.38337755]], dtype=float32),
 array([[ 0.12150974,  0.3351387 , -0.44303304, ..., -0.09911355,
          0.4791212 ,  0.6913995 ],
        [-0.1986731 ,  0.03576344, -0.45409974, ..., -0.48704138,
          0.4756418 ,  0.33339462],
        [ 0.13500328, -0.14053795, -0.27462244, ..., -0.47922388,
          0.29712686,  0.65227866],
        ...,
        [ 0.04077114,  0.40602347,  0.2

#### Удалим ненужные более массивы и запустим сборщик мусора

In [31]:
del padded, attention_mask
gc.collect()

43

#### Выделим признаки

In [32]:
features = np.concatenate(embeddings)

In [33]:
features.shape

(36900, 768)

In [34]:
target_downsampled.shape

(36904,)

#### Размерности матрицы признаков и матрицы целевого признака отличаются ввиду цпецифики проведения эмбеддинга, исключим из матрицы целевого признака лишние значения с конца

In [66]:
index = list(range(0, features.shape[0]))

In [70]:
target_downsampled = target_downsampled[index]

In [72]:
target_downsampled.shape

(36900,)

#### Разделим выборку на тренировочную и тестовую

In [73]:
features_train, features_test, target_train, target_test = train_test_split(
    features, 
    target_downsampled, 
    test_size=0.3)

#### Мы будем проводить обучение методом кросс-валидации, поэтому выделение отдельно валидационной выборки не нужно

## Выводы по шагу 1:
    1. Данные загружены и проанализированы.
    2. Целевой признак приведен к типу int8, для уменьшения затрат ресурсов.
    3. Текст приведен к нижнему регистру.
    4. Текст токенизирован при помощи BertTokenizerFast.
    5. Из датасета исключены векторы длинной больше 512.
    6. Выборка сбалансирована методом downsampling.
    7. Сформированы векторы равной длины (padded) и маска признаков.
    8. При помощи BertModel проведен эмбеддинг ткстов.
    9. Выборка разделена на тренировочную и тестовую.
    10. Данные готовы к обучению моделей.

## 2. Обучение

#### Обучение будем проводить моделями логистической регрессии, дерева решений, случайного леса и бэггинга решающих деревьев.

In [90]:
def cr_val_sc(model, features, target):
    """
    Функция проводит кросс-валидацию и считает среднюю метрику
    """
    score = cross_val_score(model, features, target, cv=5, scoring = 'f1')
    final_score = score.mean()
    return print('Величина f1-score по результатам кросс-валидации составила: ',final_score)

In [91]:
model_logistic = LogisticRegression(solver='sag')
model_forest = RandomForestClassifier(random_state=12345, n_jobs=-1)
model_tree =  DecisionTreeClassifier(random_state=12345)
model_bagging = BaggingClassifier(random_state=12345, n_jobs=-1)

#### Кросс-валидация модели Логистической Регресии

In [92]:
cr_val_sc(model_logistic, features_train, target_train)

Величина f1-score по результатам кросс-валидации составила:  0.8755172062749829


#### Зададим параметры для кросс-валидации

In [93]:
params_forest = {
    'n_estimators': list(range(50,300,50)),
    'max_depth':[5,15],
    'max_features' : list(range(1,20, 2))
}

In [3]:
estim = list(range(50, 300, 50))
estim

[50, 100, 150, 200, 250]

In [94]:
params_tree = {   
    'max_depth':list(range(1,20))  
}

In [95]:
params_bagging ={
    'n_estimators': list(range(50,300,50)),
    'max_features' : list(range(1,20)),     
}

#### Обучение модели решающих деревьев

In [96]:
CV_tree = GridSearchCV(model_tree, 
                       param_grid=params_tree, 
                       scoring = 'f1', cv=5)

CV_tree.fit(features_train, target_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=DecisionTreeClassifier(class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort=False, random_state=12345,
                                              splitter='best'),
             iid='warn', n_jobs=None, param_grid={'max_depth': [1, 20]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='f1

In [97]:
print('Лучшее значение f1_score:', CV_tree.best_score_)
print('С параметром', CV_tree.best_params_)

Лучшее значение f1_score: 0.7160962760727237
С параметром {'max_depth': 20}


#### Обучение модели случайного леса

In [98]:
CV_forest = GridSearchCV(model_forest, 
                         param_grid=params_forest, 
                         scoring='f1', 
                         cv=5)

CV_forest.fit(features_train, target_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators='warn', n_jobs=-1,
                                              oob_score=False,
                                              random_state=12345, verbose=0,
                                              warm_start=False),
             iid=

In [99]:
print('Лучшее значение f1_score:', CV_forest.best_score_)
print('С параметром', CV_forest.best_params_)

Лучшее значение f1_score: 0.8271542106517517
С параметром {'max_depth': 15, 'max_features': 20, 'n_estimators': 300}


#### Обучение модели Бэггинга

In [100]:
CV_bagging = GridSearchCV(model_bagging, 
                          param_grid=params_bagging, 
                          scoring= 'f1', 
                          cv=5)
CV_bagging.fit(features_train, target_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=BaggingClassifier(base_estimator=None, bootstrap=True,
                                         bootstrap_features=False,
                                         max_features=1.0, max_samples=1.0,
                                         n_estimators=10, n_jobs=-1,
                                         oob_score=False, random_state=12345,
                                         verbose=0, warm_start=False),
             iid='warn', n_jobs=None,
             param_grid={'max_features': [1, 20],
                         'n_estimators': [50, 300, 50]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='f1', verbose=0)

In [101]:
print('Лучшее значение f1_score:', CV_bagging.best_score_)
print('С параметром', CV_bagging.best_params_)

Лучшее значение f1_score: 0.80120621757899
С параметром {'max_features': 20, 'n_estimators': 300}


#### Лучшие результаты показала модель логистической регрессии, обучим её и проверим на тестовой выборке

In [102]:
model_final = LogisticRegression()

In [103]:
model_final.fit(features_train, target_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [104]:
prediction_train = model_final.predict(features_train)
print('f1_score для тренировочной выборки составил ', f1_score(target_train, prediction_train))

f1_score для тренировочной выборки составил  0.8916591115140525


In [105]:
prediction_test = model_final.predict(features_test)
print('f1_score для тестовой выборки составил ', f1_score(target_test, prediction_test))

f1_score для тестовой выборки составил  0.8761258874642366


## Выводы по шагу 2:
    1. Для решения задачи были использованы модели логистической регрессии, случайного леса, дерева решений и бэггинга.
    2. Проведена кросс-валидация результатов предсказаний.
    3. Для моделей подобраны наилучшие параметры.
    4. Наилучшие результаты показала логистическая регрессия со средним результатом f1_score = 0,875.
    5. Наилучшая модель обучена на тренировочной выборке.
    6. F1_score на тестовой выборке показал результат 0,876.

# 3. Выводы

#### Проведем проверку на адекватность

In [117]:
from sklearn.dummy import DummyClassifier
model_dummy = DummyClassifier()

In [118]:
model_dummy.fit(features_train, target_train)
prediction_dummy = model_dummy.predict(features_test)
print('f1_score для Dummy модели составил ', f1_score(target_test, prediction_dummy))

f1_score для Dummy модели составил  0.43100189035916825


    1. В ходе работы были проведена предобработка исходного датасета средствами BERT модели.
    2. Для обучения были выбраны модели LogisticRegression, RandomForest, DescissionTree, Bagging.
    3. Был проведен подбор наилучших параметров и кросс-валидация моделей на тренировочной выборке.
    4. Кросс-валидация показала следующие результаты f1_score на тренировочной выборке:
        - LogisticRegression - 0.875
        - RandomForest - 0.827
        - DescissionTree - 0.716
        - Bagging - 0.8
    5. Все модели кроме решающих деревьев показали результаты лучше заданных - более 0.75.
    6. Проверка модели на адекватность при помощи DummyClassifier прошла успешно. f1_score для DUmyy модели - 0.43.
    7. Лучшая модель LogisticRegression на тестовой выборке показала результат f1_score = 0.876.